In [1]:
import os

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

# librosa is a Python library for analyzing audio and music.
# It can be used to extract the data from the audio files
import librosa 
import librosa.display
import audioread
# to play the audio files
from IPython.display import Audio
plt.style.use('seaborn-white')
import soundfile

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

import sys

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
RAV = "/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/"

In [3]:
dir = os.listdir(RAV)

males = []
females = [] 
        
for actor in dir:
       
    files = os.listdir(RAV + actor)
        
    for file in files: 
        part = file.split('.')[0]
        part = part.split("-")           
            
        temp = int(part[6])        
                
        if part[2] == '01':
            emotion = 'neutral'
        elif part[2] == '02':
            emotion = 'calm'
        elif part[2] == '03':
            emotion = 'happy'
        elif part[2] == '04':
            emotion = 'sad'
        elif part[2] == '05':
            emotion = 'angry'
        elif part[2] == '06':
            emotion = 'fear'
        elif part[2] == '07':
            emotion = 'disgust'
        elif part[2] == '08':
            emotion = 'surprise'
        else:
            emotion = 'unknown'
            
        if temp%2 == 0:
            path = (RAV + actor + '/' + file)
            #emotion = 'female_'+emotion
            females.append([emotion, path]) 
        else:
            path = (RAV + actor + '/' + file)
             #emotion = 'male_'+emotion
            males.append([emotion, path])   
    
   
RavFemales_df = pd.DataFrame(females)
RavFemales_df.columns = ['Emotion', 'Path']

RavMales_df = pd.DataFrame(males)
RavMales_df.columns = ['Emotion', 'Path']

print('RAVDESS datasets')
RavFemales_df.head()


RAVDESS datasets


,Emotion,Path
0,angry,/Users/kokilareddy/Downloads/download/audio_sp...
1,fear,/Users/kokilareddy/Downloads/download/audio_sp...
2,fear,/Users/kokilareddy/Downloads/download/audio_sp...
3,angry,/Users/kokilareddy/Downloads/download/audio_sp...
4,disgust,/Users/kokilareddy/Downloads/download/audio_sp...


In [5]:
RavFemales_df['Gender']='Female'
RavMales_df['Gender']='Male'

In [6]:
Audiodata=RavFemales_df.append(RavMales_df, ignore_index=True)

In [7]:
Audiodata

,Emotion,Path,Gender
0,angry,/Users/kokilareddy/Downloads/download/audio_sp...,Female
1,fear,/Users/kokilareddy/Downloads/download/audio_sp...,Female
2,fear,/Users/kokilareddy/Downloads/download/audio_sp...,Female
3,angry,/Users/kokilareddy/Downloads/download/audio_sp...,Female
4,disgust,/Users/kokilareddy/Downloads/download/audio_sp...,Female
...,...,...,...
1435,neutral,/Users/kokilareddy/Downloads/download/audio_sp...,Male
1436,calm,/Users/kokilareddy/Downloads/download/audio_sp...,Male
1437,calm,/Users/kokilareddy/Downloads/download/audio_sp...,Male
1438,happy,/Users/kokilareddy/Downloads/download/audio_sp...,Male


In [8]:
Audiodata.Emotion.unique() 

array(['angry', 'fear', 'disgust', 'sad', 'surprise', 'happy', 'calm',
       'neutral'], dtype=object)

In [9]:
Audiodata['Paths']=Audiodata['Path']+"@"+Audiodata['Emotion']
Paths=Audiodata['Paths']

In [10]:
# LSTM
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import RMSprop

In [11]:
def extract_mfcc(wav_file_name):
    #This function extracts mfcc features and obtain the mean of each dimension
    #Input : path_to_wav_file
    #Output: mfcc_features'''
    data,sr=librosa.load(wav_file_name , res_type='kaiser_fast')
    result = np.array([])    
    mfccs = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=40)
    mfccs_processed = np.mean(mfccs.T,axis=0)
    return mfccs_processed

Audio_labels = []
Audio_data = []
for file in Paths:
    print(file)
    File=file.split("@")[0]
    emotion=file.split("@")[1]
    Audio_data.append(extract_mfcc(File)) # extract MFCC features/file
    if(emotion == "angry"):
        Audio_labels.append([0])
    elif(emotion == "neutral"):
        Audio_labels.append([1])
    elif(emotion == "disgust"):
        Audio_labels.append([2])
    elif(emotion == "sad"):
        Audio_labels.append([3])
    elif(emotion == "fear"):
        Audio_labels.append([4])
    elif(emotion == "happy"):
        Audio_labels.append([5])
    elif(emotion == "surprise"):
        Audio_labels.append([6])
    else:
        Audio_labels.append([7])

/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_16/03-01-05-01-02-01-16.wav@angry
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_16/03-01-06-01-02-02-16.wav@fear
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_16/03-01-06-02-01-02-16.wav@fear
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_16/03-01-05-02-01-01-16.wav@angry
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_16/03-01-07-01-01-01-16.wav@disgust
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_16/03-01-04-01-01-02-16.wav@sad
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_16/03-01-04-02-02-02-16.wav@sad
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_16/03-01-07-02-02-01-16.wav@disgust
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_16/03-01-08-02-02-01-16.wav@surprise
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24

/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_18/03-01-07-01-01-02-18.wav@disgust
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_18/03-01-04-01-01-01-18.wav@sad
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_18/03-01-04-02-02-01-18.wav@sad
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_18/03-01-07-02-02-02-18.wav@disgust
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_18/03-01-08-02-02-02-18.wav@surprise
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_18/03-01-08-01-01-02-18.wav@surprise
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_18/03-01-03-02-02-01-18.wav@happy
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_18/03-01-03-01-01-01-18.wav@happy
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_18/03-01-02-02-01-02-18.wav@calm
/Users/kokilareddy/Downloads/download/audio_speech_actors_0

/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_20/03-01-05-01-01-02-20.wav@angry
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_20/03-01-05-02-02-02-20.wav@angry
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_20/03-01-06-02-02-01-20.wav@fear
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_20/03-01-04-01-02-01-20.wav@sad
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_20/03-01-07-01-02-02-20.wav@disgust
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_20/03-01-07-02-01-02-20.wav@disgust
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_20/03-01-04-02-01-01-20.wav@sad
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_20/03-01-08-02-02-01-20.wav@surprise
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_20/03-01-08-01-01-01-20.wav@surprise
/Users/kokilareddy/Downloads/download/audio_speech_actors_0

/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_10/03-01-08-02-01-01-10.wav@surprise
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_04/03-01-03-02-02-02-04.wav@happy
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_04/03-01-03-01-01-02-04.wav@happy
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_04/03-01-02-02-01-01-04.wav@calm
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_04/03-01-01-01-02-02-04.wav@neutral
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_04/03-01-02-01-02-01-04.wav@calm
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_04/03-01-05-01-02-01-04.wav@angry
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_04/03-01-06-01-02-02-04.wav@fear
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_04/03-01-06-02-01-02-04.wav@fear
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24

/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_02/03-01-02-01-01-02-02.wav@calm
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_02/03-01-05-01-01-02-02.wav@angry
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_02/03-01-06-01-01-01-02.wav@fear
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_02/03-01-06-02-02-01-02.wav@fear
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_02/03-01-05-02-02-02-02.wav@angry
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_02/03-01-07-01-02-02-02.wav@disgust
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_02/03-01-04-01-02-01-02.wav@sad
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_02/03-01-04-02-01-01-02.wav@sad
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_02/03-01-07-02-01-02-02.wav@disgust
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Act

...

/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_05/03-01-05-02-02-02-05.wav@angry
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_05/03-01-05-01-01-02-05.wav@angry
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_05/03-01-06-01-01-01-05.wav@fear
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_05/03-01-08-01-02-02-05.wav@surprise
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_05/03-01-08-02-01-02-05.wav@surprise
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_05/03-01-01-01-01-01-05.wav@neutral
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_05/03-01-02-01-01-02-05.wav@calm
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_05/03-01-02-02-02-02-05.wav@calm
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_05/03-01-03-01-02-01-05.wav@happy
/Users/kokilareddy/Downloads/download/audio_speech_actors_0

/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_15/03-01-06-02-01-01-15.wav@fear
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_15/03-01-05-02-01-02-15.wav@angry
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_15/03-01-05-01-02-02-15.wav@angry
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_15/03-01-06-01-02-01-15.wav@fear
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_15/03-01-08-01-02-01-15.wav@surprise
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_15/03-01-08-02-01-01-15.wav@surprise
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_15/03-01-07-02-01-01-15.wav@disgust
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_15/03-01-04-02-01-02-15.wav@sad
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_15/03-01-04-01-02-02-15.wav@sad
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-2

/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_13/03-01-02-02-01-01-13.wav@calm
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_13/03-01-01-01-02-02-13.wav@neutral
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_13/03-01-02-01-02-01-13.wav@calm
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_13/03-01-08-02-02-01-13.wav@surprise
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_13/03-01-08-01-01-01-13.wav@surprise
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_13/03-01-05-01-02-01-13.wav@angry
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_13/03-01-06-01-02-02-13.wav@fear
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_13/03-01-06-02-01-02-13.wav@fear
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_13/03-01-05-02-01-01-13.wav@angry
/Users/kokilareddy/Downloads/download/audio_speech_actors_01

/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_09/03-01-08-01-01-02-09.wav@surprise
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_09/03-01-06-01-02-01-09.wav@fear
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_09/03-01-05-01-02-02-09.wav@angry
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_09/03-01-05-02-01-02-09.wav@angry
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_09/03-01-06-02-01-01-09.wav@fear
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_09/03-01-04-01-01-01-09.wav@sad
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_09/03-01-07-01-01-02-09.wav@disgust
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_09/03-01-07-02-02-02-09.wav@disgust
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_09/03-01-04-02-02-01-09.wav@sad
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24

/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_07/03-01-02-01-02-02-07.wav@calm
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_07/03-01-02-02-01-02-07.wav@calm
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_07/03-01-03-01-01-01-07.wav@happy
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_07/03-01-03-02-02-01-07.wav@happy
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_07/03-01-02-01-01-01-07.wav@calm
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_07/03-01-01-01-01-02-07.wav@neutral
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_07/03-01-02-02-02-01-07.wav@calm
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_07/03-01-03-01-02-02-07.wav@happy
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Actor_07/03-01-03-02-01-02-07.wav@happy
/Users/kokilareddy/Downloads/download/audio_speech_actors_01-24/Ac

In [12]:
Audio_labels_array = np.array(Audio_labels)
Audio_data_array = np.array(Audio_data)

In [13]:
Audio_data_array.shape,Audio_labels_array.shape

((1440, 40), (1440, 1))

In [14]:
labels_categorical = to_categorical(Audio_labels_array)
labels_categorical

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [15]:
x_train,x_test,y_train,y_test= train_test_split(np.array(Audio_data_array),
                                                labels_categorical,
                                                test_size=0.20,
                                                random_state=9)

In [16]:
# Split the training, validating, and testing sets
number_of_samples = Audio_data_array.shape[0]
training_samples = int(number_of_samples * 0.8)
validation_samples = int(number_of_samples * 0.1)
test_samples = int(number_of_samples * 0.1)

In [17]:
# making our data compatible to model.
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1152, 40, 1), (1152, 8), (288, 40, 1), (288, 8))

In [18]:
def create_model_LSTM():
    model = Sequential()
    model.add(LSTM(units = 128, return_sequences = True, input_shape = (x_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 128, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 64, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 64))
    model.add(Dropout(0.2))
    model.add(Dense(units = 32))
    model.add(Dense(64))
    model.add(Dropout(0.4))
    model.add(Activation('relu'))
    model.add(Dense(32))
    model.add(Dropout(0.4))
    model.add(Activation('relu'))
    model.add(Dense(8))
    model.add(Activation('softmax'))

    model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics=['accuracy'])
    return model

In [19]:
model_A = create_model_LSTM()

In [25]:
history=model_A.fit(x_train,y_train,
    epochs=150, shuffle=True , batch_size = 128)

Epoch 1/150
9/9 [==============================] - 7s 778ms/step - loss: 0.2280 - accuracy: 0.5920
Epoch 2/150
9/9 [==============================] - 7s 745ms/step - loss: 0.2327 - accuracy: 0.5694
Epoch 3/150
9/9 [==============================] - 7s 816ms/step - loss: 0.2246 - accuracy: 0.5903
Epoch 4/150
9/9 [==============================] - 6s 675ms/step - loss: 0.2266 - accuracy: 0.5773
Epoch 5/150
9/9 [==============================] - 6s 661ms/step - loss: 0.2239 - accuracy: 0.5842
Epoch 6/150
9/9 [==============================] - 6s 630ms/step - loss: 0.2095 - accuracy: 0.6189
Epoch 7/150
9/9 [==============================] - 10s 1s/step - loss: 0.2074 - accuracy: 0.6224
Epoch 8/150
9/9 [==============================] - 8s 894ms/step - loss: 0.2099 - accuracy: 0.6345
Epoch 9/150
9/9 [==============================] - 5s 598ms/step - loss: 0.2047 - accuracy: 0.6233
Epoch 10/150
9/9 [==============================] - 6s 714ms/step - loss: 0.2100 - accuracy: 0.6076
Epoch 11/15

9/9 [==============================] - 5s 589ms/step - loss: 0.1004 - accuracy: 0.8516
Epoch 84/150
9/9 [==============================] - 7s 740ms/step - loss: 0.1019 - accuracy: 0.8481
Epoch 85/150
9/9 [==============================] - 7s 746ms/step - loss: 0.0943 - accuracy: 0.8550
Epoch 86/150
9/9 [==============================] - 5s 582ms/step - loss: 0.1075 - accuracy: 0.8273
Epoch 87/150
9/9 [==============================] - 5s 564ms/step - loss: 0.1101 - accuracy: 0.8281
Epoch 88/150
9/9 [==============================] - 5s 554ms/step - loss: 0.1188 - accuracy: 0.8255
Epoch 89/150
9/9 [==============================] - 5s 553ms/step - loss: 0.1147 - accuracy: 0.8082
Epoch 90/150
9/9 [==============================] - 5s 538ms/step - loss: 0.1058 - accuracy: 0.8559
Epoch 91/150
9/9 [==============================] - 5s 552ms/step - loss: 0.0846 - accuracy: 0.8845
Epoch 92/150
9/9 [==============================] - 7s 787ms/step - loss: 0.0910 - accuracy: 0.8707
Epoch 93/150


In [21]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1152, 40, 1), (1152, 8), (288, 40, 1), (288, 8))

In [22]:
path_ = "/Users/kokilareddy/Downloads/download/AudioWAV//1037_ITS_HAP_XX.wav"

import IPython.display as ipd
ipd.Audio(path_)
a = extract_mfcc(path_)
a1 = np.asarray(a)
q = np.expand_dims(a1,1)
qq = np.expand_dims(q,0)
pred = model_A.predict(qq)
preds=pred.argmax(axis=1)
print(preds[0])
if preds[0]==0:
    print("Angry")
elif preds[0]==1:
    print("Neutral")
elif preds[0]==2:
    print("Disgust")
elif preds[0]==3:
    print("Sad")
elif preds[0]==4:
    print("Fear")
elif preds[0]==5:
    print("Happy")
elif preds[0]==6:
    print("Surprise")
elif preds[0]==7:
    print("Calm")

2
Disgust


In [28]:
# predicting on test data.
pred_test = model_A.predict(x_test)
y_pred = pred_test.argmax(axis=1)
y_test = y_test.argmax(axis=1)

In [29]:
# - Calculate the accuracy of our model
accuracy=accuracy_score(y_test, y_pred )

# - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 44.79%


In [30]:
import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write 
# import wavio as wv
import scipy.io.wavfile as wav

fs=44100
duration = 5  # seconds
myrecording = sd.rec(duration * fs, samplerate=fs, channels=2,dtype='float64')
print("Recording Audio")
sd.wait()
print("Audio recording complete ,checking emotion")
write("recording0.wav", fs, myrecording)
path_="recording0.wav"
import IPython.display as ipd
ipd.Audio(path_)
a = extract_mfcc(path_)
a1 = np.asarray(a)
q = np.expand_dims(a1,1)
qq = np.expand_dims(q,0)
pred = model_A.predict(qq)
preds=pred.argmax(axis=1)
print(preds[0])
if preds[0]==0:
    print("Angry")
elif preds[0]==1:
    print("Neutral")
elif preds[0]==2:
    print("Disgust")
elif preds[0]==3:
    print("Sad")
elif preds[0]==4:
    print("Fear")
elif preds[0]==5:
    print("Happy")
elif preds[0]==6:
    print("Surprise")
elif preds[0]==7:
    print("Calm")

Recording Audio
Audio recording complete ,checking emotion
0
Angry
